In [44]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from random import randint
from time import sleep
from lxml import etree

In [45]:
target_page = [9, 5, 48, 7, 4, 12, 10, 11, 13, 14, 16, 37, 3, 49]

def get_data(page):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    soup = BeautifulSoup(driver.page_source,"lxml")
    driver.get('https://www.taiwannews.com.tw/en/cate/{}'.format(str(page)))
    button = driver.find_elements(by=By.XPATH, value="//button[@class='btn btn-more center-block']")[0]
    button.click()
    sleep(randint(1,10))
    button.click()
    sleep(randint(1,10))
    button.click()
    sleep(randint(1,10))
    button.click()
    sleep(randint(1,10))
    soup = BeautifulSoup(driver.page_source,"lxml")
    dom = etree.HTML(str(soup))

    table = soup.find('table', {'class': 'table mod_table-fixed-width'})

    other_dates = [i.text for i in dom.xpath("//div[@class='entry-date']/span[2]")]
    other_urls = ["https://www.taiwannews.com.tw/"+i for i in dom.xpath("//a[@title='READ MORE >']/@href")]
    other_titles = [td.text.replace('\n', '') for td in soup.find_all('h3', {'class': 'entry-title'})]
    other_list = [list(ele) for ele in list(zip(other_titles, other_dates, other_urls))]

    trs = table.find_all('tr')[1:]
    rows = list()
    extra_rows = list()
    for tr in trs:
        rows.append([td.text for td in tr.find_all('td')[1:]])
        extra_rows.append(['https://www.taiwannews.com.tw'+td['href'] for td in tr.find_all('a', href=True)][0])
    for i in range(len(rows)):
        rows[i].append(extra_rows[i])
    for i in range(5):
        rows.append(other_list[i])
    df = pd.DataFrame(rows,
                  columns=['title', 'date', 'url'])
    return df

In [49]:
df = get_data(target_page[0])
for i in target_page[1:]:
    sleep(randint(1,10))
    df = pd.concat([df, get_data(i)], ignore_index=True)

/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_45286/3932687540.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_45286/3932687540.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_45286/3932687540.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_45286/3932687540.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_45286/3932687

In [51]:
df['platform'] = 'taiwan-news'
df.to_csv('taiwan-news.csv', encoding='utf-8-sig', columns=['platform', 'date', 'url', 'title'])
tmp = pd.read_csv('taiwan-news.csv', index_col=0)
tmp

,platform,date,url,title
0,taiwan-news,2022/12/15 12:12,https://www.taiwannews.com.tw/en/news/4751184,"MEP Anna Fotyga in Taiwan to discuss economic,..."
1,taiwan-news,2022/12/15 11:03,https://www.taiwannews.com.tw/en/news/4751118,"Taiwanese American nonprofit donates US$600,00..."
2,taiwan-news,2022/12/15 09:20,https://www.taiwannews.com.tw/en/news/4751046,"Taiwan tracks 9 Chinese military aircraft, 5 n..."
3,taiwan-news,2022/12/14 19:55,https://www.taiwannews.com.tw/en/news/4750516,Taiwan works to increase fines for drivers not...
4,taiwan-news,2022/12/14 19:25,https://www.taiwannews.com.tw/en/news/4750310,China’s food product recipe requirement applie...
...,...,...,...,...
718,taiwan-news,2022/12/15 11:56,https://www.taiwannews.com.tw//en/news/4751162,Taiwan and Korea's CSD chiefs meet in Taipei t...
719,taiwan-news,2022/10/01 17:50,https://www.taiwannews.com.tw//en/news/4674207,"Far Eastern New Century wins multiple awards, ..."
720,taiwan-news,2022/09/30 16:59,https://www.taiwannews.com.tw//en/news/4673286,Mandarin Oriental Taipei awarded Best City Hot...
721,taiwan-news,2022/09/08 17:47,https://www.taiwannews.com.tw//en/news/4652129,Taiwan paper company markets 100% recyclable f...
